In [18]:
import nibabel as nib
import numpy as np

ct_path = r"Z:\Angie\SMILE_facialdeformation\StJude_cohort\abby\1720903\Plan\1720903_plan_shifted.nii"
mandible_mask_path = r"Z:\Angie\SMILE_facialdeformation\StJude_cohort\abby\1720903\Plan\TotalSegmentator1\mandible.nii.gz"
mirror_axis = 0
out_prefix = r"Z:\Angie\SMILE_facialdeformation\StJude_cohort\abby\1720903\Plan\mirror"

ct_nib = nib.load(ct_path)
mask_nib = nib.load(mandible_mask_path)

print("CT shape:", ct_nib.shape)
print("CT affine:\n", ct_nib.affine)

def mirror_nib_image(img, axis=0):
    data = img.get_fdata()
    flipped = np.flip(data, axis=axis).copy()

    affine = img.affine.copy()
    dims = img.shape

    translation = affine[:3, axis] * (dims[axis] - 1)

    new_affine = affine.copy()
    new_affine[:3, axis] *= -1
    new_affine[:3, 3] = affine[:3, 3] + translation

    return nib.Nifti1Image(flipped, new_affine, img.header)

ct_mirrored = mirror_nib_image(ct_nib, axis=mirror_axis)
mask_mirrored = mirror_nib_image(mask_nib, axis=mirror_axis)

ct_mirrored_path = out_prefix + "_ct.nii.gz"
mask_mirrored_path = out_prefix + "_mandible.nii.gz"

nib.save(ct_mirrored, ct_mirrored_path)
nib.save(mask_mirrored, mask_mirrored_path)

print("Saved:", ct_mirrored_path)
print("Saved:", mask_mirrored_path)


CT shape: (512, 512, 179)
CT affine:
 [[-8.12492371e-01  0.00000000e+00 -0.00000000e+00  2.07589996e+02]
 [ 0.00000000e+00 -8.12492371e-01 -1.69007988e-10  4.63939972e+02]
 [ 0.00000000e+00  6.86585927e-11 -2.00000763e+00 -4.13500000e+02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
Saved: Z:\Angie\SMILE_facialdeformation\StJude_cohort\abby\1720903\Plan\mirror_ct.nii.gz
Saved: Z:\Angie\SMILE_facialdeformation\StJude_cohort\abby\1720903\Plan\mirror_mandible.nii.gz


In [19]:
%matplotlib inline

import matplotlib.pyplot as plt
import nibabel as nib
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output
import os

# ---------------------- User paths ----------------------
ct1_path = r"Z:\Angie\SMILE_facialdeformation\StJude_cohort\abby\1720903\Plan\1720903_plan_shifted.nii"
ct2_path = r"Z:\Angie\SMILE_facialdeformation\StJude_cohort\abby\1720903\Plan\mirror_ct.nii.gz"
# ---------------------------------------------------------

# ---- Load CTs ----
ct1_img = nib.load(ct1_path)
ct2_img = nib.load(ct2_path)

ct1 = ct1_img.get_fdata()
ct2 = ct2_img.get_fdata()

# Normalise for viewing
ct1 = (ct1 - np.min(ct1)) / (np.max(ct1) - np.min(ct1))
ct2 = (ct2 - np.min(ct2)) / (np.max(ct2) - np.min(ct2))

spacing = ct1_img.header.get_zooms()

# Colormap for CT2 overlay
overlay_cmap = plt.cm.bone   # pink/yellow-ish
overlay_cmap.set_under(alpha=0)  # invisible for low values


# ---------------------- Slice Viewer ----------------------
def show_slice(slice_index, plane, alpha=0.35):

    plt.figure(figsize=(6, 6))

    # ---------------- Axial ----------------
    if plane == 'Axial':
        img1 = np.rot90(ct1[:, :, slice_index])
        img2 = np.rot90(ct2[:, :, slice_index])
        aspect = spacing[1] / spacing[0]

    # ---------------- Coronal ----------------
    elif plane == 'Coronal':
        img1 = np.flipud(np.rot90(ct1[:, slice_index, :]))
        img2 = np.flipud(np.rot90(ct2[:, slice_index, :]))
        aspect = spacing[2] / spacing[0]

    # ---------------- Sagittal ----------------
    elif plane == 'Sagittal':
        img1 = np.flipud(np.rot90(ct1[slice_index, :, :]))
        img2 = np.flipud(np.rot90(ct2[slice_index, :, :]))
        aspect = spacing[2] / spacing[1]

    # ---- Base CT ----
    plt.imshow(img1, cmap='gray', origin='lower', aspect=aspect)

    # ---- Overlay CT ----
    overlay = overlay_cmap(img2)
    overlay[..., 3] = alpha   # set transparency
    plt.imshow(overlay, origin='lower', aspect=aspect)

    plt.title(f"{plane} slice {slice_index}")
    plt.axis("off")

    clear_output(wait=True)
    display(plt.gcf())
    plt.close()


# ---------------------- Widgets ----------------------
plane_dropdown = widgets.Dropdown(
    options=['Axial', 'Coronal', 'Sagittal'],
    value='Axial',
    description='Plane:'
)

slice_slider = widgets.IntSlider(
    value=ct1.shape[2] // 2,
    min=0,
    max=ct1.shape[2] - 1,
    step=1,
    description='Slice:',
    continuous_update=True,
    layout=widgets.Layout(width='80%')
)

alpha_slider = widgets.FloatSlider(
    value=0.35,
    min=0.0,
    max=1.0,
    step=0.05,
    description='Overlay opacity:',
    continuous_update=True,
    layout=widgets.Layout(width='60%')
)


# update slice range when plane changes
def update_slider_range(*args):
    plane = plane_dropdown.value
    if plane == 'Axial':
        slice_slider.max = ct1.shape[2] - 1
        slice_slider.value = ct1.shape[2] // 2
    elif plane == 'Coronal':
        slice_slider.max = ct1.shape[1] - 1
        slice_slider.value = ct1.shape[1] // 2
    elif plane == 'Sagittal':
        slice_slider.max = ct1.shape[0] - 1
        slice_slider.value = ct1.shape[0] // 2

plane_dropdown.observe(update_slider_range, names='value')
update_slider_range()

widgets.interact(
    show_slice,
    slice_index=slice_slider,
    plane=plane_dropdown,
    alpha=alpha_slider
)


interactive(children=(IntSlider(value=89, description='Slice:', layout=Layout(width='80%'), max=178), Dropdown…

<function __main__.show_slice(slice_index, plane, alpha=0.35)>

In [24]:
import SimpleITK as sitk
import numpy as np

orig_ct_path = r"Z:\Angie\SMILE_facialdeformation\StJude_cohort\abby\1720903\Plan\1720903_plan_shifted.nii"
mirrored_ct_path = r"Z:\Angie\SMILE_facialdeformation\StJude_cohort\abby\1720903\Plan\mirror_ct.nii.gz"

orig = sitk.ReadImage(orig_ct_path, sitk.sitkFloat32)
mir = sitk.ReadImage(mirrored_ct_path, sitk.sitkFloat32)

print("Original:", orig.GetSize(), "Mirrored:", mir.GetSize())


registration_method = sitk.ImageRegistrationMethod()
registration_method.SetMetricAsMattesMutualInformation(numberOfHistogramBins=50)
registration_method.SetMetricSamplingPercentage(0.50, sitk.sitkWallClock)
registration_method.SetMetricSamplingStrategy(registration_method.RANDOM)

registration_method.SetInterpolator(sitk.sitkLinear)

registration_method.SetOptimizerAsGradientDescent(
    learningRate=1.0,
    numberOfIterations=300,
    convergenceMinimumValue=1e-6,
    convergenceWindowSize=10
)
registration_method.SetOptimizerScalesFromPhysicalShift()

initial_transform = sitk.Euler3DTransform()  # translation + small rotations only
initial_transform.SetCenter(center.tolist())
registration_method.SetInitialTransform(initial_transform, inPlace=False)

final_transform = registration_method.Execute(orig, mir)

print("Final metric value:", registration_method.GetMetricValue())
print("Optimizer stop condition:", registration_method.GetOptimizerStopConditionDescription())
print("Final transform:")
print(final_transform)

resampled_mirror = sitk.Resample(
    mir,
    orig,
    final_transform,
    sitk.sitkLinear,
    0.0,
    mir.GetPixelID()
)

sitk.WriteImage(resampled_mirror, r"Z:\Angie\SMILE_facialdeformation\StJude_cohort\abby\1720903\Plan\mirrored_registered_ct.nii.gz")
sitk.WriteTransform(final_transform, r"Z:\Angie\SMILE_facialdeformation\StJude_cohort\abby\1720903\Plan\mirror_to_orig_rigid.tfm")

print("Rigidly registered CT saved.")


Original: (512, 512, 179) Mirrored: (512, 512, 179)
Final metric value: -0.2983846108722047
Optimizer stop condition: GradientDescentOptimizerv4Template: Convergence checker passed at iteration 9.
Final transform:
itk::simple::CompositeTransform
 CompositeTransform (00000198BF44EF00)
   RTTI typeinfo:   class itk::CompositeTransform<double,3>
   Reference Count: 1
   Modified Time: 168938141
   Debug: Off
   Object Name: 
   Observers: 
     none
   TransformQueue: 
   >>>>>>>>>
   Euler3DTransform (000001992F19A7A0)
     RTTI typeinfo:   class itk::Euler3DTransform<double>
     Reference Count: 1
     Modified Time: 168937987
     Debug: Off
     Object Name: 
     Observers: 
       none
     Matrix: 
       1 -3.96945e-05 -0.000159515 
       3.98409e-05 1 0.000917846 
       0.000159479 -0.000917852 1 
     Offset: [0.418401, -6.69503, 0.0664261]
     Center: [0.408051, -255.942, -592.501]
     Translation: [0.523073, -7.23873, 0.301665]
     Inverse: 
       1 3.98409e-05 0.000159

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import nibabel as nib
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output
import os

# ---------------------- User paths ----------------------
ct1_path = r"Z:\Angie\SMILE_facialdeformation\StJude_cohort\abby\1720903\Plan\1720903_plan_shifted.nii"
ct2_path = r"Z:\Angie\SMILE_facialdeformation\StJude_cohort\abby\1720903\Plan\mirrored_registered_ct.nii.gz"
# ---------------------------------------------------------

# ---- Load CTs ----
ct1_img = nib.load(ct1_path)
ct2_img = nib.load(ct2_path)

ct1 = ct1_img.get_fdata()
ct2 = ct2_img.get_fdata()

# Normalise for viewing
ct1 = (ct1 - np.min(ct1)) / (np.max(ct1) - np.min(ct1))
ct2 = (ct2 - np.min(ct2)) / (np.max(ct2) - np.min(ct2))

spacing = ct1_img.header.get_zooms()

# Colormap for CT2 overlay
overlay_cmap = plt.cm.bone 
overlay_cmap.set_under(alpha=0)  # invisible for low values


# ---------------------- Slice Viewer ----------------------
def show_slice(slice_index, plane, alpha=0.35):

    plt.figure(figsize=(6, 6))

    # ---------------- Axial ----------------
    if plane == 'Axial':
        img1 = np.rot90(ct1[:, :, slice_index])
        img2 = np.rot90(ct2[:, :, slice_index])
        aspect = spacing[1] / spacing[0]

    # ---------------- Coronal ----------------
    elif plane == 'Coronal':
        img1 = np.flipud(np.rot90(ct1[:, slice_index, :]))
        img2 = np.flipud(np.rot90(ct2[:, slice_index, :]))
        aspect = spacing[2] / spacing[0]

    # ---------------- Sagittal ----------------
    elif plane == 'Sagittal':
        img1 = np.flipud(np.rot90(ct1[slice_index, :, :]))
        img2 = np.flipud(np.rot90(ct2[slice_index, :, :]))
        aspect = spacing[2] / spacing[1]

    # ---- Base CT ----
    plt.imshow(img1, cmap='gray', origin='lower', aspect=aspect)

    # ---- Overlay CT ----
    overlay = overlay_cmap(img2)
    overlay[..., 3] = alpha   # set transparency
    plt.imshow(overlay, origin='lower', aspect=aspect)

    plt.title(f"{plane} slice {slice_index}")
    plt.axis("off")

    clear_output(wait=True)
    display(plt.gcf())
    plt.close()


# ---------------------- Widgets ----------------------
plane_dropdown = widgets.Dropdown(
    options=['Axial', 'Coronal', 'Sagittal'],
    value='Axial',
    description='Plane:'
)

slice_slider = widgets.IntSlider(
    value=ct1.shape[2] // 2,
    min=0,
    max=ct1.shape[2] - 1,
    step=1,
    description='Slice:',
    continuous_update=True,
    layout=widgets.Layout(width='80%')
)

alpha_slider = widgets.FloatSlider(
    value=0.35,
    min=0.0,
    max=1.0,
    step=0.05,
    description='Overlay opacity:',
    continuous_update=True,
    layout=widgets.Layout(width='60%')
)


# update slice range when plane changes
def update_slider_range(*args):
    plane = plane_dropdown.value
    if plane == 'Axial':
        slice_slider.max = ct1.shape[2] - 1
        slice_slider.value = ct1.shape[2] // 2
    elif plane == 'Coronal':
        slice_slider.max = ct1.shape[1] - 1
        slice_slider.value = ct1.shape[1] // 2
    elif plane == 'Sagittal':
        slice_slider.max = ct1.shape[0] - 1
        slice_slider.value = ct1.shape[0] // 2

plane_dropdown.observe(update_slider_range, names='value')
update_slider_range()

widgets.interact(
    show_slice,
    slice_index=slice_slider,
    plane=plane_dropdown,
    alpha=alpha_slider
)


interactive(children=(IntSlider(value=89, description='Slice:', layout=Layout(width='80%'), max=178), Dropdown…

<function __main__.show_slice(slice_index, plane, alpha=0.35)>

In [ ]:
import SimpleITK as sitk

img = sitk.ReadImage(r"Z:\Angie\SMILE_facialdeformation\StJude_cohort\abby\1720903\Plan\1720903_plan_shifted.nii")

# Mirror across LR axis by reversing the first dimension
flipped_array = sitk.GetArrayFromImage(img)[:, :, ::-1]  # flip x-axis
img_flipped = sitk.GetImageFromArray(flipped_array)
img_flipped.CopyInformation(img)  # preserve spacing, origin, direction

sitk.WriteImage(img_flipped, r"Z:\Angie\SMILE_facialdeformation\StJude_cohort\abby\1720903\Plan\1720903_plan_mirrored.nii")
print("saved")


In [29]:
%matplotlib inline

import matplotlib.pyplot as plt
import nibabel as nib
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output
import os

# ---------------------- User paths ----------------------
ct1_path = r"Z:\Angie\SMILE_facialdeformation\StJude_cohort\abby\1720903\Plan\1720903_plan_shifted.nii"
ct2_path = r"Z:\Angie\SMILE_facialdeformation\StJude_cohort\abby\1720903\Plan\1720903_plan_mirrored.nii"
# ---------------------------------------------------------

# ---- Load CTs ----
ct1_img = nib.load(ct1_path)
ct2_img = nib.load(ct2_path)

ct1 = ct1_img.get_fdata()
ct2 = ct2_img.get_fdata()

# Normalise for viewing
ct1 = (ct1 - np.min(ct1)) / (np.max(ct1) - np.min(ct1))
ct2 = (ct2 - np.min(ct2)) / (np.max(ct2) - np.min(ct2))

spacing = ct1_img.header.get_zooms()

# Colormap for CT2 overlay
overlay_cmap = plt.cm.bone   # pink/yellow-ish
overlay_cmap.set_under(alpha=0)  # invisible for low values


# ---------------------- Slice Viewer ----------------------
def show_slice(slice_index, plane, alpha=0.35):

    plt.figure(figsize=(6, 6))

    # ---------------- Axial ----------------
    if plane == 'Axial':
        img1 = np.rot90(ct1[:, :, slice_index])
        img2 = np.rot90(ct2[:, :, slice_index])
        aspect = spacing[1] / spacing[0]

    # ---------------- Coronal ----------------
    elif plane == 'Coronal':
        img1 = np.flipud(np.rot90(ct1[:, slice_index, :]))
        img2 = np.flipud(np.rot90(ct2[:, slice_index, :]))
        aspect = spacing[2] / spacing[0]

    # ---------------- Sagittal ----------------
    elif plane == 'Sagittal':
        img1 = np.flipud(np.rot90(ct1[slice_index, :, :]))
        img2 = np.flipud(np.rot90(ct2[slice_index, :, :]))
        aspect = spacing[2] / spacing[1]

    # ---- Base CT ----
    plt.imshow(img1, cmap='gray', origin='lower', aspect=aspect)

    # ---- Overlay CT ----
    overlay = overlay_cmap(img2)
    overlay[..., 3] = alpha   # set transparency
    plt.imshow(overlay, origin='lower', aspect=aspect)

    plt.title(f"{plane} slice {slice_index}")
    plt.axis("off")

    clear_output(wait=True)
    display(plt.gcf())
    plt.close()


# ---------------------- Widgets ----------------------
plane_dropdown = widgets.Dropdown(
    options=['Axial', 'Coronal', 'Sagittal'],
    value='Axial',
    description='Plane:'
)

slice_slider = widgets.IntSlider(
    value=ct1.shape[2] // 2,
    min=0,
    max=ct1.shape[2] - 1,
    step=1,
    description='Slice:',
    continuous_update=True,
    layout=widgets.Layout(width='80%')
)

alpha_slider = widgets.FloatSlider(
    value=0.35,
    min=0.0,
    max=1.0,
    step=0.05,
    description='Overlay opacity:',
    continuous_update=True,
    layout=widgets.Layout(width='60%')
)


# update slice range when plane changes
def update_slider_range(*args):
    plane = plane_dropdown.value
    if plane == 'Axial':
        slice_slider.max = ct1.shape[2] - 1
        slice_slider.value = ct1.shape[2] // 2
    elif plane == 'Coronal':
        slice_slider.max = ct1.shape[1] - 1
        slice_slider.value = ct1.shape[1] // 2
    elif plane == 'Sagittal':
        slice_slider.max = ct1.shape[0] - 1
        slice_slider.value = ct1.shape[0] // 2

plane_dropdown.observe(update_slider_range, names='value')
update_slider_range()

widgets.interact(
    show_slice,
    slice_index=slice_slider,
    plane=plane_dropdown,
    alpha=alpha_slider
)


interactive(children=(IntSlider(value=89, description='Slice:', layout=Layout(width='80%'), max=178), Dropdown…

<function __main__.show_slice(slice_index, plane, alpha=0.35)>

In [ ]:
import SimpleITK as sitk

fixed = sitk.ReadImage(r"Z:\Angie\SMILE_facialdeformation\StJude_cohort\abby\1720903\Plan\1720903_plan_shifted.nii")   
moving = sitk.ReadImage(r"Z:\Angie\SMILE_facialdeformation\StJude_cohort\abby\1720903\Plan\1720903_plan_mirrored.nii")   

initial_tx = sitk.CenteredTransformInitializer(
    fixed,
    moving,
    sitk.VersorRigid3DTransform(),
    sitk.CenteredTransformInitializerFilter.GEOMETRY
)

reg = sitk.ImageRegistrationMethod()

reg.SetMetricAsMattesMutualInformation(numberOfHistogramBins=50)
reg.SetMetricSamplingPercentage(0.2, sitk.sitkWallClock)
reg.SetMetricSamplingStrategy(reg.RANDOM)

reg.SetInterpolator(sitk.sitkLinear)

reg.SetOptimizerAsRegularStepGradientDescent(
    learningRate=2.0,
    minStep=1e-4,
    numberOfIterations=200,
    gradientMagnitudeTolerance=1e-6
)

# prevents big rotations that cause the optimizer to flip
reg.SetOptimizerScalesFromPhysicalShift()

reg.SetInitialTransform(initial_tx, inPlace=False)

final_tx = reg.Execute(fixed, moving)

sitk.WriteTransform(final_tx, r"Z:\Angie\SMILE_facialdeformation\StJude_cohort\abby\1720903\Plan\rigid_mirror_to_original.tfm")
print("Saved registration transform")

resampler = sitk.ResampleImageFilter()

resampler.SetReferenceImage(fixed)
resampler.SetInterpolator(sitk.sitkLinear)
resampler.SetTransform(final_tx)

registered_mirror_CT = resampler.Execute(moving)

sitk.WriteImage(registered_mirror_CT, r"Z:\Angie\SMILE_facialdeformation\StJude_cohort\abby\1720903\Plan\mirror_registered_ct.nii.gz")
print("Saved registered CT")


In [31]:
resampler = sitk.ResampleImageFilter()

resampler.SetReferenceImage(fixed)
resampler.SetInterpolator(sitk.sitkLinear)
resampler.SetTransform(final_tx)

registered_mirror_CT = resampler.Execute(moving)

sitk.WriteImage(registered_mirror_CT, r"Z:\Angie\SMILE_facialdeformation\StJude_cohort\abby\1720903\Plan\mirror_registered_ct.nii.gz")


In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import nibabel as nib
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output
import os

# ---------------------- User paths ----------------------
ct1_path = r"Z:\Angie\SMILE_facialdeformation\StJude_cohort\abby\1720903\Plan\1720903_plan_shifted.nii"
ct2_path = r"Z:\Angie\SMILE_facialdeformation\StJude_cohort\abby\1720903\Plan\mirror_registered_ct.nii.gz"
# ---------------------------------------------------------

# ---- Load CTs ----
ct1_img = nib.load(ct1_path)
ct2_img = nib.load(ct2_path)

ct1 = ct1_img.get_fdata()
ct2 = ct2_img.get_fdata()

# Normalise for viewing
ct1 = (ct1 - np.min(ct1)) / (np.max(ct1) - np.min(ct1))
ct2 = (ct2 - np.min(ct2)) / (np.max(ct2) - np.min(ct2))

spacing = ct1_img.header.get_zooms()

# Colormap for CT2 overlay
overlay_cmap = plt.cm.bone   # pink/yellow-ish
overlay_cmap.set_under(alpha=0)  # invisible for low values


# ---------------------- Slice Viewer ----------------------
def show_slice(slice_index, plane, alpha=0.35):

    plt.figure(figsize=(6, 6))

    # ---------------- Axial ----------------
    if plane == 'Axial':
        img1 = np.rot90(ct1[:, :, slice_index])
        img2 = np.rot90(ct2[:, :, slice_index])
        aspect = spacing[1] / spacing[0]

    # ---------------- Coronal ----------------
    elif plane == 'Coronal':
        img1 = np.flipud(np.rot90(ct1[:, slice_index, :]))
        img2 = np.flipud(np.rot90(ct2[:, slice_index, :]))
        aspect = spacing[2] / spacing[0]

    # ---------------- Sagittal ----------------
    elif plane == 'Sagittal':
        img1 = np.flipud(np.rot90(ct1[slice_index, :, :]))
        img2 = np.flipud(np.rot90(ct2[slice_index, :, :]))
        aspect = spacing[2] / spacing[1]

    # ---- Base CT ----
    plt.imshow(img1, cmap='gray', origin='lower', aspect=aspect)

    # ---- Overlay CT ----
    overlay = overlay_cmap(img2)
    overlay[..., 3] = alpha   # set transparency
    plt.imshow(overlay, origin='lower', aspect=aspect)

    plt.title(f"{plane} slice {slice_index}")
    plt.axis("off")

    clear_output(wait=True)
    display(plt.gcf())
    plt.close()


# ---------------------- Widgets ----------------------
plane_dropdown = widgets.Dropdown(
    options=['Axial', 'Coronal', 'Sagittal'],
    value='Axial',
    description='Plane:'
)

slice_slider = widgets.IntSlider(
    value=ct1.shape[2] // 2,
    min=0,
    max=ct1.shape[2] - 1,
    step=1,
    description='Slice:',
    continuous_update=True,
    layout=widgets.Layout(width='80%')
)

alpha_slider = widgets.FloatSlider(
    value=0.35,
    min=0.0,
    max=1.0,
    step=0.05,
    description='Overlay opacity:',
    continuous_update=True,
    layout=widgets.Layout(width='60%')
)


# update slice range when plane changes
def update_slider_range(*args):
    plane = plane_dropdown.value
    if plane == 'Axial':
        slice_slider.max = ct1.shape[2] - 1
        slice_slider.value = ct1.shape[2] // 2
    elif plane == 'Coronal':
        slice_slider.max = ct1.shape[1] - 1
        slice_slider.value = ct1.shape[1] // 2
    elif plane == 'Sagittal':
        slice_slider.max = ct1.shape[0] - 1
        slice_slider.value = ct1.shape[0] // 2

plane_dropdown.observe(update_slider_range, names='value')
update_slider_range()

widgets.interact(
    show_slice,
    slice_index=slice_slider,
    plane=plane_dropdown,
    alpha=alpha_slider
)


interactive(children=(IntSlider(value=89, description='Slice:', layout=Layout(width='80%'), max=178), Dropdown…

<function __main__.show_slice(slice_index, plane, alpha=0.35)>